# Building a WebScaper to gather Data of Books Selling on Amazon.in

#### This Jupyter Notebook was used to test and write code, Final program can also be ran from 'main.py'

In [16]:
#importing BeautifulSoup 4 and selenium
from bs4 import BeautifulSoup
import selenium
import pandas as pd

In [17]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [18]:
import pandas as pd

### Start the webdriver

In [19]:
# startup the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\boraw\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
<ipython-input-19-a8f3e5a54fdc>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [19]:
url = "https://www.amazon.in/"
driver.get(url)

In [20]:
def get_url(search_term):
    """Generates a URL from the search term."""
    generic_url = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ','+') # conforming to the amazon's url format for search terms with scapes
   
    url = generic_url.format(search_term)
    
    # added page query
    url += '&page={}'
    
    return url

In [145]:
url = get_url('Books')
url

'https://www.amazon.in/s?k=Books&page={}'

In [146]:
driver.get(url)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x01117413+2389011]
	Ordinal0 [0x010A9F61+1941345]
	Ordinal0 [0x00F9C520+836896]
	Ordinal0 [0x00F90682+788098]
	Ordinal0 [0x00F90EB8+790200]
	Ordinal0 [0x00F92752+796498]
	Ordinal0 [0x00F8C0D9+770265]
	Ordinal0 [0x00F9D9D0+842192]
	Ordinal0 [0x00FF3AE2+1194722]
	Ordinal0 [0x00FE3F66+1130342]
	Ordinal0 [0x00FBE546+976198]
	Ordinal0 [0x00FBF456+980054]
	GetHandleVerifier [0x012C9632+1727522]
	GetHandleVerifier [0x0137BA4D+2457661]
	GetHandleVerifier [0x011AEB81+569713]
	GetHandleVerifier [0x011ADD76+566118]
	Ordinal0 [0x010B0B2B+1968939]
	Ordinal0 [0x010B5988+1989000]
	Ordinal0 [0x010B5A75+1989237]
	Ordinal0 [0x010BECB1+2026673]
	BaseThreadInitThunk [0x75AE6739+25]
	RtlGetFullPathName_UEx [0x778B8E7F+1215]
	RtlGetFullPathName_UEx [0x778B8E4D+1165]


### Extract the Collection

In [23]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [25]:
results = soup.find_all(class_= "a-section a-spacing-small s-padding-left-small s-padding-right-small")

In [26]:
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [27]:
len(results)

22

In [28]:
item = results[0]

In [29]:
atag = item.h2.a

In [30]:
title = atag.text.strip()
title

'World’s Greatest Books For Personal Growth & Wealth (Set of 4 Books): Perfect Motivational Gift Set'

In [31]:
product_url = "https:amazon.in" + atag.get('href')
product_url

'https:amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A10239661C8CJHMGI2PI7&qualifier=1650794074&id=8732908638006251&widgetName=sp_atf&url=%2FWorlds-Greatest-Personal-Growth-Wealth%2Fdp%2F9389432014%2Fref%3Dsr_1_1_sspa%3Fkeywords%3DBooks%26qid%3D1650794074%26sr%3D8-1-spons%26psc%3D1'

In [32]:
price_parent = item.find('span','a-price')

In [33]:
price_parent.find('span','a-offscreen').text[1:]

'325'

In [34]:
rating = item.i.text
rating

'4.5 out of 5 stars'

In [38]:
rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
rating_count

'18,417'

In [37]:
author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
author

AttributeError: 'NoneType' object has no attribute 'text'

### General Function for getting details

In [237]:
def extract_details(item):
    """Extracts details of a single record"""
    atag = item.h2.a 
    title = atag.text.strip()
    
    product_url = "https://amazon.in" + atag.get('href')
    price_parent = item.find('span','a-price')
    price = price_parent.find('span','a-offscreen').text[1:]
    rating = item.i.text
    rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
    
    try: # used try block as some author names are missing
        author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
    except AttributeError:
        author = 'unknown'
        
    output = (title,author,product_url,price,rating, rating_count)
    
    return output


In [238]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})

for item in results:
    records.append(extract_details(item))


# Put it all together

In [3]:
def get_url(search_term):
    """Generates a URL from the search term."""
    generic_url = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ','+') # conforming to the amazon's url format for search terms with scapes
   
    url = generic_url.format(search_term)
    
    # added page query
    url += '&page={}'
    return url

def extract_details(item):
    """Extracts details of a single record"""
    atag = item.h2.a 
    title = atag.text.strip()
    
    product_url = "https://amazon.in" + atag.get('href')
    price_parent = item.find('span','a-price')
    
    try: # used try block for errors due to missing values
        price = price_parent.find('span','a-offscreen').text[1:]
        author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
        rating = item.i.text
        rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
    except AttributeError:
        price = 'unkown'
        author = 'unknown'
        rating = 'unknown'
        rating_count = 'unknown'
        
    output = (title,author,product_url,price,rating, rating_count)
    return output


def main(search_term):
    """Run Main program"""
    # startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    records = []
    url = get_url(search_term)
    
    for page in range(1,3):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_details(item)
            if record:
                records.append(record)
    
    driver.close()
    global df
    df = pd.DataFrame(records)
    df.columns = ['Title','Author','Link','Price',"Rating",'Rating Count']
    
    
    
    
    #df.to_csv('amazon_books_records.csv', sep=',')


In [291]:
url = get_url('Books')
url

'https://www.amazon.in/s?k=Books&page={}'

In [51]:
for page in range(1,21):
    print(url.format(page))

https://www.amazon.in/s?k=Books&page=1
https://www.amazon.in/s?k=Books&page=2
https://www.amazon.in/s?k=Books&page=3
https://www.amazon.in/s?k=Books&page=4
https://www.amazon.in/s?k=Books&page=5
https://www.amazon.in/s?k=Books&page=6
https://www.amazon.in/s?k=Books&page=7
https://www.amazon.in/s?k=Books&page=8
https://www.amazon.in/s?k=Books&page=9
https://www.amazon.in/s?k=Books&page=10
https://www.amazon.in/s?k=Books&page=11
https://www.amazon.in/s?k=Books&page=12
https://www.amazon.in/s?k=Books&page=13
https://www.amazon.in/s?k=Books&page=14
https://www.amazon.in/s?k=Books&page=15
https://www.amazon.in/s?k=Books&page=16
https://www.amazon.in/s?k=Books&page=17
https://www.amazon.in/s?k=Books&page=18
https://www.amazon.in/s?k=Books&page=19
https://www.amazon.in/s?k=Books&page=20


## Crawling on the collectec product links to extract more information

In [44]:
driver.get(level1_url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [66]:
description = soup.find('div',{"class":"a-expander-content a-expander-partial-collapse-content"}).text.strip()
description

"A collection of world's timeless classics, this box set includes the four greatest bestsellers, which have inspired readers for generations. Packed with wisdom and time-tested principles that are as relevant in modern times as ever before, these inspirational books are a must-read for all those aspiring for personal growth and wealth."

In [83]:
product_details = soup.find('div',{"id":"detailBullets_feature_div"})
product_details

<div class="celwidget" data-csa-c-id="pgp6dy-b8fflh-uq65a1-qbt2cv" data-feature-name="detailBullets" id="detailBullets_feature_div">
<style type="text/css">
    .detail-bullets-wrapper ul.detail-bullet-list {
        margin: 0 0 1px 18px;
    }
    .detail-bullets-wrapper ul li {
        margin-bottom: 5.5px;
    }
    .detail-bullets-wrapper:last-child {
        margin-bottom: 4.5px;
    }
</style>
<div class="a-section feature detail-bullets-wrapper bucket" data-feature-name="detailBullets" data-template-name="detailBullets" id="detailBulletsWrapper_feature_div"> <hr aria-hidden="true" class="a-divider-normal bucketDivider"/> <h2>Product details</h2>
<div id="detailBullets_feature_div">
<ul class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"> <li><span class="a-list-item"> <span class="a-text-bold">Publisher
                                    ‏
                                        :
                                    ‎
                                

In [109]:
Language = product_details.find_all('span',{'class':'a-list-item'})[1].find_all('span')[1].text.strip()
Language

'English'

In [149]:
def get_url(search_term):
    """Generates a URL from the search term."""
    generic_url = 'https://www.amazon.in/s?k={}'
    search_term = search_term.replace(' ','+') # conforming to the amazon's url format for search terms with scapes
   
    url = generic_url.format(search_term)
    
    # added page query
    url += '&page={}'
    return url

def extract_details(item):
    """Extracts details of a single record"""
    atag = item.h2.a 
    title = atag.text.strip()
    
    product_url = "https://amazon.in" + atag.get('href')
    price_parent = item.find('span','a-price')
    
    try: # used try block for errors due to missing values
        price = price_parent.find('span','a-offscreen').text[1:]
        author = item.find('a',{'class':'a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style'}).text.strip()
        rating = item.i.text
        rating_count  = item.find('span',{"class":"a-size-base s-underline-text"}).text
    except AttributeError:
        price = 'unkown'
        author = 'unknown'
        rating = 'unknown'
        rating_count = 'unknown'
        
    output = (title,author,product_url,price,rating, rating_count)
    return output


def main(search_term, level):
    """Run Main program,
        level: 0:Search Page Results, 1 : Product Page Results"""
    # startup the webdriver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_details(item)
            if record:
                records.append(record)

            
    global df
    df = pd.DataFrame(records)
    df.columns = ['Title','Author','Link','Price',"Rating",'Rating Count']
    
    if level>0:
        ##additional code for next level depth crawling and scraping product details, product description
        list_language = []
        list_book_description = []
        for i in range(len(df['Link'])):
            driver.get(df['Link'][i])
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')
            book_description = soup2.find('div',{"class":"a-expander-content a-expander-partial-collapse-content"}).text.strip()
            product_details = soup2.find('div',{"id":"detailBullets_feature_div"})
            language = product_details.find_all('span',{'class':'a-list-item'})[1].find_all('span')[1].text.strip()

            list_language.append(language)
            list_book_description.append(book_description)

        df['Langauge'] = list_language
        df['Description'] = list_book_description
        
    driver.close()
    df.to_csv('amazon_books_records_with_description.csv', sep=',')


In [150]:
from datetime import datetime
start_time = datetime.now()

#main('Books',0) # for product search results only
main('Books',1) # for product search results and individual product details

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\boraw\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
<ipython-input-149-48d34a043a58>:39: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x01117413+2389011]
	Ordinal0 [0x010A9F61+1941345]
	Ordinal0 [0x00F9C520+836896]
	Ordinal0 [0x00F90682+788098]
	Ordinal0 [0x00F90EB8+790200]
	Ordinal0 [0x00F92752+796498]
	Ordinal0 [0x00F8C0D9+770265]
	Ordinal0 [0x00F9D9D0+842192]
	Ordinal0 [0x00FF3AE2+1194722]
	Ordinal0 [0x00FE3F66+1130342]
	Ordinal0 [0x00FBE546+976198]
	Ordinal0 [0x00FBF456+980054]
	GetHandleVerifier [0x012C9632+1727522]
	GetHandleVerifier [0x0137BA4D+2457661]
	GetHandleVerifier [0x011AEB81+569713]
	GetHandleVerifier [0x011ADD76+566118]
	Ordinal0 [0x010B0B2B+1968939]
	Ordinal0 [0x010B5988+1989000]
	Ordinal0 [0x010B5A75+1989237]
	Ordinal0 [0x010BECB1+2026673]
	BaseThreadInitThunk [0x75AE6739+25]
	RtlGetFullPathName_UEx [0x778B8E7F+1215]
	RtlGetFullPathName_UEx [0x778B8E4D+1165]


In [147]:
df.head()

,Title,Author,Link,Price,Rating,Rating Count,Langauge,Description
0,World’s Greatest Books For Personal Growth & W...,unknown,https://amazon.in/gp/slredirect/picassoRedirec...,unkown,unknown,unknown,English,"A collection of world's timeless classics, thi..."
1,3 and a Half Murders: An Inspector Saralkar My...,Salil Desai,https://amazon.in/gp/slredirect/picassoRedirec...,179,4.3 out of 5 stars,776,English,Two corpses . . . a woman lying dead on her be...
2,The Legend of Bahirji-Naik: Raiders of Surat (...,unknown,https://amazon.in/Legend-Bahirji-Naik-Raiders-...,unkown,unknown,unknown,English,"Maratha Swarajya, 1663: The dream of a kingdom..."
3,The Psychology of Money,Morgan Housel,https://amazon.in/Psychology-Money-Morgan-Hous...,300,4.6 out of 5 stars,"32,300",English,"Timeless lessons on wealth, greed, and happine..."
4,Life's Amazing Secrets: How to Find Balance an...,Gaur Gopal Das,https://amazon.in/Lifes-Amazing-Secrets-Balanc...,221,4.6 out of 5 stars,"15,140",English,"Stop going through life, Start growing through..."


---END OF NOTEBOOK ---